In [35]:
import pandas as pd
import os

# Set dir
thisfile_dir = os.getcwd()
#thisfile_dir = os.path.dirname(os.path.abspath(__file__))
dir_cur = os.path.join(thisfile_dir, '..','outputs','extracted_data')
print("Target Directory:", dir_cur)

xlsx_file = "singtel-sustainability-report-2021.pdf.xlsx"
input_file = os.path.join(f"{dir_cur}", xlsx_file)

summary_file = "Summary_table.xlsx"
summary_path = os.path.join(f"{dir_cur}",'..' , summary_file)
summary = pd.read_excel(summary_path)

data = pd.read_excel(input_file)

unique_units = data['unit'].unique()
print("Unique values in 'unit' column:", unique_units)
print(summary)

Target Directory: d:\GitHub\DSS5105_BugBuster\tests\..\outputs\extracted_data
Unique values in 'unit' column: ['tCO2e' 'MWhs' 'ML' 't' 'Male Percentage (%)' 'Female Percentage (%)'
 'Millennials (%)' 'Percentage (%)' 'Hours/No. of employees'
 'Number of cases' 'Discussion and number' 'List' 'External' 'tCO2e/year'
 'tCO2e/TB' 'Number' 'm³' 'Description' 'Number and Percentage (%)'
 'Internal' 'Gen Z (%)' 'Male Hours/No. of employees' 'Baby Boomers (%)'
 'Gen Xers (%)' 'GJ' 'Million dollars' 'Hours' 'People' 'Male Percentage'
 'Female Percentage' 'Millennials Percentage' 'Percentage'
 'Female Hours/No. of employees']
   Company Name  Year  GHG Emissions (Scope 1) (tCO2e)  \
0         Test2  2012                              NaN   
1         Test2  2013                              NaN   
2         Test2  2015                              NaN   
3         Test2  2016                              NaN   
4         Test2  2017                              NaN   
5         Test2  2018       

In [ ]:
exchange_rates = {
    'EUR': 1.1,   # 1 EUR = 1.1 USD
    'GBP': 1.3,   
    'CAD': 0.8,   
    'CNY': 0.139,  
    'SGD': 0.754,
    'JPY': 0.0091, 
    'AUD': 0.7,
    'USD': 1  
}

def modify_units(row):
        # Check if the 'year' field is not null
    if pd.notnull(row['year']):
        year_str = str(row['year'])
        
        # Initialize variables for finding the year sequence
        potential_year = ''
        year_found = False
        
        for char in year_str:
            if char.isdigit():
                potential_year += char
            else:
                # Reset if the sequence goes beyond four digits
                potential_year = ''

            # Once four consecutive digits are found, validate
            if len(potential_year) == 4:
                year_int = int(potential_year)
                # Define the valid year range
                min_year, max_year = 2015, 2025
                if min_year <= year_int <= max_year:
                    row['year'] = year_int
                    year_found = True
                    break
                else:
                    # Reset and continue searching if invalid
                    potential_year = ''

        if not year_found:
            row['year'] = None
    else:
        row['year'] = None
    
    # Existing unit modifications
    if row['unit'] == 'GJ':
        row['unit'] = 'MWhs'
        row['value'] = row['value'] * 0.277778  # GJ to MWh
    elif row['unit'] == 'm³':
        row['unit'] = 'ML'
        row['value'] = row['value'] * 0.001  # m3 to ML
    elif row['unit'] == 'm3':
        row['unit'] = 'ML'
        row['value'] = row['value'] * 0.001  # m3 to ML
    elif row['unit'] == 'kg':
        row['unit'] = 't'
        row['value'] = row['value'] / 1000  # kg to t
    elif row['unit'] in exchange_rates:
        row['value'] = row['value'] * exchange_rates[row['unit']]
        row['unit'] = 'USD' 
    elif row['unit'] == 'Million dollars':
        row['value'] = row['value'] * 1000000
        row['unit'] = 'USD' 

    return row

data = data.apply(modify_units, axis=1)
data

,Unnamed: 0,label,metric,unit,year,value
0,1,Greenhouse Gas Emissions,Total,tCO2e,2021,15000
1,2,Energy Consumption,Total energy consumption,MWhs,2021,250000
2,3,Water Consumption,Total water consumption,ML,2021,500
3,4,Waste Generation,Total waste generated,t,2021,1000
4,5,Gender Diversity,Current employees by gender,Male Percentage (%),2021,60
...,...,...,...,...,...,...
372,404,Occupational Health & Safety,High-consequence injuries,Number of cases,2020,5
373,405,Occupational Health & Safety,Recordable injuries,Number of cases,2020,20
374,406,Recordable work-related illnesses,Number of recordable work-related illnesses or...,Number of cases,2020,8
375,407,Development & Training,Average training hours per employee by gender,Male Hours/No. of employees,2020,30


In [37]:
def fill_esg_data(df, new_df, column_name, label=None, metric=None, unit=None):
    condition = pd.Series([True] * len(df))
    if label is not None:
        condition &= (df['label'].str.contains(label, regex=False))
    if metric is not None:
        condition &= (df['metric'].str.contains(metric, regex=False))
    if unit is not None:
        condition &= (df['unit'].str.contains(unit, regex=False))
    
    filtered_df = df[condition]

    def custom_agg(values):
        numeric_values = pd.to_numeric(values, errors='coerce').dropna()
        string_values = values[numeric_values.index.difference(values.index)]

        if not numeric_values.empty: 
            return numeric_values.max() 
        else:
            return ', '.join(string_values.unique()) 

    max_values_by_year = filtered_df.groupby('year')['value'].agg(custom_agg).reset_index()
    max_values_by_year = max_values_by_year.set_index('year')

    if column_name not in new_df.columns:
        new_df[column_name] = ''

    for year, row in max_values_by_year.iterrows():
        new_df.loc[year, column_name] = row['value']




def restructure(df,company_name):
    new_df = pd.DataFrame(columns=['year'])
    new_df.set_index('year', inplace=True)
    
    fill_esg_data(df, new_df, 'GHG Emissions (Scope 1) (tCO2e)', label='Greenhouse Gas Emissions', metric='Scope 1', unit = 'tCO2e')
    fill_esg_data(df, new_df, 'GHG Emissions (Scope 2) (tCO2e)', label='Greenhouse Gas Emissions', metric='Scope 2', unit='tCO2e')
    fill_esg_data(df, new_df, 'GHG Emissions (Scope 3) (tCO2e)', label='Greenhouse Gas Emissions', metric='Scope 3', unit='tCO2e')
    fill_esg_data(df, new_df, 'GHG Emissions (Total) (tCO2e)', label='Greenhouse Gas Emissions', metric='Total', unit='tCO2e')
    fill_esg_data(df, new_df, 'Total Energy Consumption (MWhs)', label='Energy Consumption', metric='Total energy consumption', unit='MWhs')
    fill_esg_data(df, new_df, 'Total Water Consumption (ML)', label='Water Consumption', metric='Total water consumption', unit='ML')
    fill_esg_data(df, new_df, 'Total Waste Generated (t)', label='Waste Generation', metric='Total waste generated', unit='t')
    fill_esg_data(df, new_df, 'Current Employees by Gender (Female %)', label='Gender Diversity', metric='Current employees by gender', unit='Female Percentage (%)')
    fill_esg_data(df, new_df, 'New Hires and Turnover by Gender (Female %)', label='Gender Diversity', metric='New hires and turnover by gender', unit='Female Percentage (%)')
    fill_esg_data(df, new_df, 'Current Employees by Age Groups (Millennials %)', label='Age-Based Diversity', metric='Current employees by age groups', unit='Millennials (%)')
    fill_esg_data(df, new_df, 'New Hires and Turnover by Age Groups (Millennials %)', label='Age-Based Diversity', metric='New hires and turnover by age groups', unit='Millennials (%)')
    fill_esg_data(df, new_df, 'Total Turnover (%)', label='Employment', metric='Total employee turnover')
    fill_esg_data(df, new_df, 'Total Number of Employees', label='Employment', metric='Total number of employees')
    fill_esg_data(df, new_df, 'Average Training Hours per Employee', label='Development & Training', metric='Average training hours per employee')
    fill_esg_data(df, new_df, 'Fatalities', metric='Fatalities')
    fill_esg_data(df, new_df, 'High-consequence injuries', metric='High-consequence injuries')
    fill_esg_data(df, new_df, 'Recordable injuries', metric='Recordable injuries')
    fill_esg_data(df, new_df, 'Recordable work-related ill health cases', metric='Number of recordable work-related illnesses or health conditions')
    fill_esg_data(df, new_df, 'Board Independence (%)', label='Board Composition', metric='Board independence')
    fill_esg_data(df, new_df, 'Women on the Board (%)', label='Board Composition', metric='Women on the board')
    fill_esg_data(df, new_df, 'Women in Management Team (%)', label='Management Diversity', metric='Women in the management team')
    fill_esg_data(df, new_df, 'Anti-Corruption Disclosures', metric='Anti-corruption disclosures')
    fill_esg_data(df, new_df, 'Anti-Corruption Training for Employees (%)', label='Ethical Behaviour', metric='Anti-corruption training for employees')
    fill_esg_data(df, new_df, 'List of Relevant Certifications', label='Certifications', metric='List of relevant certifications')
    fill_esg_data(df, new_df, 'Alignment with Frameworks and Disclosure Practices', label='Alignment with Frameworks', metric='Alignment with frameworks and disclosure practices')
    fill_esg_data(df, new_df, 'Assurance of Sustainability Report', label='Assurance', metric='Assurance of sustainability report')

    new_df.insert(0, 'Company Name', company_name)
    new_df.rename_axis('Year', inplace=True)
    new_df.reset_index(inplace=True)
    new_df.fillna('', inplace=True)

    return new_df
    
new_df = restructure(data,'Company Name')
print(new_df)


    Year  Company Name GHG Emissions (Scope 1) (tCO2e)  \
0   2012  Company Name                                   
1   2013  Company Name                                   
2   2015  Company Name                                   
3   2016  Company Name                                   
4   2017  Company Name                                   
5   2018  Company Name                                   
6   2019  Company Name                          6652.0   
7   2020  Company Name                          5881.0   
8   2021  Company Name                         50000.0   
9   2022  Company Name                         10200.0   
10  2030  Company Name                                   
11  2050  Company Name                                   

   GHG Emissions (Scope 2) (tCO2e) GHG Emissions (Scope 3) (tCO2e)  \
0                                                                    
1                                                                    
2                                  

In [38]:
summary_table_path = summary_path

def append_to_summary(summary_table_path, new_df):
    existing_df = pd.read_excel(summary_table_path, sheet_name='E')

    # Ensure 'Year' and 'Company Name' are present in new_df
    if 'Year' not in new_df.columns or 'Company Name' not in new_df.columns:
        raise ValueError("new_df must contain 'Year' and 'Company Name' columns")

    # Go through each row in new_df
    for index, new_row in new_df.iterrows():
        # Check if there is an existing row that matches the 'Year' and 'Company Name'
        match = (existing_df['Year'] == new_row['Year']) & (existing_df['Company Name'] == new_row['Company Name'])
        
        if existing_df[match].empty:
            # If there is no matching row, concatenate the new row
            existing_df = pd.concat([existing_df, pd.DataFrame([new_row])], ignore_index=True)
        else:
            for col in new_df.columns:
                existing_value = existing_df.loc[match, col].values[0]
                new_value = new_row[col]
                
                if pd.isna(existing_value):
                    existing_df.loc[match, col] = new_value
                else:
                    # If the existing value is not empty, keep the larger one
                    if pd.notna(new_value):  # Only compare if new_value is not NaN
                        existing_df.loc[match, col] = max(existing_value, new_value)

    # Write the updated dataframe back to the same Excel file
    existing_df.to_excel(summary_table_path, sheet_name='E', index=False)

In [39]:
def xlsx_to_summary(xlsx_file_path, summary_table_path, company_name):
    data = pd.read_excel(xlsx_file_path)
    data = data.apply(modify_units, axis=1)
    new_df = restructure(data, company_name)
    append_to_summary(summary_table_path, new_df)
    
xlsx_to_summary(input_file, summary_table_path, 'Test2')

C:\Users\HONOR\AppData\Local\Temp\ipykernel_21616\2172530870.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  existing_df.loc[match, col] = new_value
C:\Users\HONOR\AppData\Local\Temp\ipykernel_21616\2172530870.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  existing_df.loc[match, col] = new_value
C:\Users\HONOR\AppData\Local\Temp\ipykernel_21616\2172530870.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  existing_df.loc[match, col] = new_value
C:\Users\H